<a href="https://colab.research.google.com/github/JinLeeGG/Survival-Prediction-Model-for-AML-using-Gene-Expression-Data-from-TCGA/blob/main/Log_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Clinical data from dataset
clinical_df = pd.read_csv('/content/drive/MyDrive/Acute Myeloid Leukemia (TCGA, PanCancer Atlas)/datasets/laml_tcga_pan_can_atlas_2018/data_clinical_patient.txt', sep='\t', comment='#')
clinical_df

,PATIENT_ID,SUBTYPE,CANCER_TYPE_ACRONYM,OTHER_PATIENT_ID,AGE,SEX,AJCC_PATHOLOGIC_TUMOR_STAGE,AJCC_STAGING_EDITION,DAYS_LAST_FOLLOWUP,DAYS_TO_BIRTH,...,WEIGHT,IN_PANCANPATHWAYS_FREEZE,OS_STATUS,OS_MONTHS,DSS_STATUS,DSS_MONTHS,DFS_STATUS,DFS_MONTHS,PFS_STATUS,PFS_MONTHS
0,TCGA-AB-2989,NaN,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,No,1:DECEASED,11.999868,NaN,NaN,NaN,NaN,NaN,NaN
1,TCGA-AB-3008,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,1:DECEASED,27.024361,NaN,NaN,NaN,NaN,NaN,NaN
2,TCGA-AB-2863,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,1:DECEASED,1.019167,NaN,NaN,NaN,NaN,NaN,NaN
3,TCGA-AB-2844,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,1:DECEASED,4.010915,NaN,NaN,NaN,NaN,NaN,NaN
4,TCGA-AB-2955,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,1:DECEASED,16.076536,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TCGA-AB-2888,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,0:LIVING,10.980702,0:ALIVE OR DEAD TUMOR FREE,NaN,NaN,NaN,NaN,NaN
196,TCGA-AB-2850,NaN,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,No,1:DECEASED,2.005457,NaN,NaN,NaN,NaN,NaN,NaN
197,TCGA-AB-2971,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,1:DECEASED,26.038071,NaN,NaN,NaN,NaN,NaN,NaN
198,TCGA-AB-2840,LAML,LAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,1:DECEASED,0.920538,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
clinical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 37 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   PATIENT_ID                                  200 non-null    object 
 1   SUBTYPE                                     162 non-null    object 
 2   CANCER_TYPE_ACRONYM                         200 non-null    object 
 3   OTHER_PATIENT_ID                            0 non-null      float64
 4   AGE                                         0 non-null      float64
 5   SEX                                         0 non-null      float64
 6   AJCC_PATHOLOGIC_TUMOR_STAGE                 0 non-null      float64
 7   AJCC_STAGING_EDITION                        0 non-null      float64
 8   DAYS_LAST_FOLLOWUP                          0 non-null      float64
 9   DAYS_TO_BIRTH                               0 non-null      float64
 10  DAYS_TO_INITIA

In [4]:
rsem_df = pd.read_csv('/content/drive/MyDrive/Acute Myeloid Leukemia (TCGA, PanCancer Atlas)/datasets/laml_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem.txt', sep='\t', comment='#')
rsem_df

,Hugo_Symbol,Entrez_Gene_Id,TCGA-AB-2803-03,TCGA-AB-2805-03,TCGA-AB-2806-03,TCGA-AB-2807-03,TCGA-AB-2808-03,TCGA-AB-2810-03,TCGA-AB-2811-03,TCGA-AB-2812-03,...,TCGA-AB-3000-03,TCGA-AB-3001-03,TCGA-AB-3002-03,TCGA-AB-3005-03,TCGA-AB-3006-03,TCGA-AB-3007-03,TCGA-AB-3008-03,TCGA-AB-3009-03,TCGA-AB-3011-03,TCGA-AB-3012-03
0,NaN,100130426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,100133144,-0.324751,0.572638,9.120017,5.892374,27.703947,0.725235,-0.324751,9.213171,...,5.873973,17.871178,9.711291,2.651229,3.873774,2.077649,16.909700,29.775759,17.716369,13.389132
2,UBE2Q2P2,100134869,-0.436849,-0.436849,8.853735,7.232627,15.704500,4.647010,3.504940,6.340629,...,0.796782,8.873421,3.601122,5.918779,8.137281,6.148142,15.335492,31.344883,8.711105,6.345575
3,HMGB1P1,10357,419.651650,419.812182,533.177649,775.751273,742.181326,667.780844,148.881530,536.349581,...,537.067281,437.370073,612.111865,436.536841,420.976459,357.992147,714.199146,481.134795,373.533101,393.514467
4,NaN,10431,1062.369221,516.945075,761.582563,656.106450,795.746840,1313.733190,1701.942971,1164.930942,...,575.036703,795.457978,701.968341,976.201282,893.317880,1356.109782,811.708451,625.457507,942.257260,1334.721961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A,440590,19.587614,3.702016,2.732140,6.577307,2.303491,15.153494,0.986118,0.301440,...,2.130553,11.080261,3.977996,4.027647,1.278239,7.372156,0.301440,7.153142,20.578090,1.490848
20527,ZYG11B,79699,855.058740,1365.417710,579.510880,930.616155,1218.266026,613.910837,146.782082,938.957013,...,772.175251,837.944659,734.534134,526.357312,726.568026,776.473450,817.559831,1122.198779,882.608575,659.918435
20528,ZYX,7791,3366.161168,4322.843702,4093.187860,2673.347317,2735.320149,7437.852503,30801.742889,2879.253659,...,5022.429882,7292.157335,4632.197647,9275.469230,4294.131894,1748.967223,4022.122449,7291.924634,3411.714796,8646.670472
20529,ZZEF1,23140,2169.713046,3323.052993,3196.702352,3652.355153,2517.518799,2491.223868,2172.788319,2612.342410,...,2797.783955,4650.041218,3720.254798,3031.202112,3141.472931,3968.650664,3537.666597,4279.346589,4007.974241,4158.690264


In [5]:
# 'Hugo_Symbol' 열에 NaN이 있는 행을 삭제합니다.
rsem_df = rsem_df.dropna(subset=['Hugo_Symbol'])

# 변경된 결과를 확인합니다.
rsem_df

,Hugo_Symbol,Entrez_Gene_Id,TCGA-AB-2803-03,TCGA-AB-2805-03,TCGA-AB-2806-03,TCGA-AB-2807-03,TCGA-AB-2808-03,TCGA-AB-2810-03,TCGA-AB-2811-03,TCGA-AB-2812-03,...,TCGA-AB-3000-03,TCGA-AB-3001-03,TCGA-AB-3002-03,TCGA-AB-3005-03,TCGA-AB-3006-03,TCGA-AB-3007-03,TCGA-AB-3008-03,TCGA-AB-3009-03,TCGA-AB-3011-03,TCGA-AB-3012-03
2,UBE2Q2P2,100134869,-0.436849,-0.436849,8.853735,7.232627,15.704500,4.647010,3.504940,6.340629,...,0.796782,8.873421,3.601122,5.918779,8.137281,6.148142,15.335492,31.344883,8.711105,6.345575
3,HMGB1P1,10357,419.651650,419.812182,533.177649,775.751273,742.181326,667.780844,148.881530,536.349581,...,537.067281,437.370073,612.111865,436.536841,420.976459,357.992147,714.199146,481.134795,373.533101,393.514467
7,RNU12-2P,26823,10.313497,10.437196,4.993648,5.556779,2.676767,2.981752,1.849247,3.941664,...,3.242098,2.179112,2.207849,1.160462,3.448370,3.606808,0.087451,2.072876,0.087451,1.108288
8,SSX9P,280660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,EZHIP,340602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A,440590,19.587614,3.702016,2.732140,6.577307,2.303491,15.153494,0.986118,0.301440,...,2.130553,11.080261,3.977996,4.027647,1.278239,7.372156,0.301440,7.153142,20.578090,1.490848
20527,ZYG11B,79699,855.058740,1365.417710,579.510880,930.616155,1218.266026,613.910837,146.782082,938.957013,...,772.175251,837.944659,734.534134,526.357312,726.568026,776.473450,817.559831,1122.198779,882.608575,659.918435
20528,ZYX,7791,3366.161168,4322.843702,4093.187860,2673.347317,2735.320149,7437.852503,30801.742889,2879.253659,...,5022.429882,7292.157335,4632.197647,9275.469230,4294.131894,1748.967223,4022.122449,7291.924634,3411.714796,8646.670472
20529,ZZEF1,23140,2169.713046,3323.052993,3196.702352,3652.355153,2517.518799,2491.223868,2172.788319,2612.342410,...,2797.783955,4650.041218,3720.254798,3031.202112,3141.472931,3968.650664,3537.666597,4279.346589,4007.974241,4158.690264


In [6]:
# --- 2. 임상 데이터 전처리 ---
# 분석에 필요한 컬럼만 선택하고 결측치가 있는 환자는 제거합니다.
df_survival = clinical_df[['PATIENT_ID', 'OS_STATUS', 'OS_MONTHS']].dropna()

# OS_STATUS를 EVENT (0 또는 1)로 변환합니다.
df_survival['EVENT'] = df_survival['OS_STATUS'].str.contains('DECEASED').astype(int)

df_survival

,PATIENT_ID,OS_STATUS,OS_MONTHS,EVENT
0,TCGA-AB-2989,1:DECEASED,11.999868,1
1,TCGA-AB-3008,1:DECEASED,27.024361,1
2,TCGA-AB-2863,1:DECEASED,1.019167,1
3,TCGA-AB-2844,1:DECEASED,4.010915,1
4,TCGA-AB-2955,1:DECEASED,16.076536,1
...,...,...,...,...
194,TCGA-AB-2923,1:DECEASED,1.019167,1
195,TCGA-AB-2888,0:LIVING,10.980702,0
196,TCGA-AB-2850,1:DECEASED,2.005457,1
197,TCGA-AB-2971,1:DECEASED,26.038071,1


In [7]:
# --- 3. 유전자 발현량 데이터 전처리 ---
# 행/열을 변환하여 '환자 x 유전자' 형태로 만듭니다.
df_exp_transposed = rsem_df.set_index('Hugo_Symbol').T

# 환자 ID를 12자리로 통일합니다.
df_exp_transposed.index = df_exp_transposed.index.str[:12]

df_exp_transposed

Hugo_Symbol,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,REXO1L6P,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
Entrez_Gene_,1.001349e+08,10357.000000,26823.000000,280660.0,340602.0,388795.000000,390284.000000,391714.000000,404770.0,441362.0,...,55055.000000,11130.000000,7789.000000,158586.000000,79364.000000,440590.000000,79699.000000,7791.000000,23140.000000,26009.000000
TCGA-AB-2803,-4.368490e-01,419.651650,10.313497,NaN,NaN,-0.255074,0.608943,0.470462,NaN,NaN,...,390.452969,355.060300,28.739696,345.024058,3237.279933,19.587614,855.058740,3366.161168,2169.713046,1071.185650
TCGA-AB-2805,-4.368490e-01,419.812182,10.437196,NaN,NaN,0.726866,1.914301,2.121447,NaN,NaN,...,542.305472,538.258372,10.568935,272.496987,3818.433863,3.702016,1365.417710,4322.843702,3323.052993,1023.538577
TCGA-AB-2806,8.853735e+00,533.177649,4.993648,NaN,NaN,5.751326,2.113764,3.806505,NaN,NaN,...,712.973785,630.936871,58.536729,627.838907,3627.132791,2.732140,579.510880,4093.187860,3196.702352,1731.476606
TCGA-AB-2807,7.232627e+00,775.751273,5.556779,NaN,NaN,-0.255074,3.497864,1.154173,NaN,NaN,...,629.874241,1025.406384,130.724674,691.987438,2458.056450,6.577307,930.616155,2673.347317,3652.355153,1243.887750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-AB-3007,6.148142e+00,357.992147,3.606808,NaN,NaN,3.821171,5.813554,0.306526,NaN,NaN,...,362.003870,695.063335,98.041575,467.072930,4162.358730,7.372156,776.473450,1748.967223,3968.650664,1312.128716
TCGA-AB-3008,1.533549e+01,714.199146,0.087451,NaN,NaN,2.720137,7.148966,0.707090,NaN,NaN,...,794.867129,1591.091341,102.043613,419.006126,2807.095490,0.301440,817.559831,4022.122449,3537.666597,1030.996789
TCGA-AB-3009,3.134488e+01,481.134795,2.072876,NaN,NaN,1.728149,5.746487,5.502807,NaN,NaN,...,316.599774,617.424855,78.215539,437.136986,3918.883096,7.153142,1122.198779,7291.924634,4279.346589,1488.291222
TCGA-AB-3011,8.711105e+00,373.533101,0.087451,NaN,NaN,4.466648,1.739097,2.050568,NaN,NaN,...,459.253913,967.510660,42.378276,353.787779,4444.331665,20.578090,882.608575,3411.714796,4007.974241,1351.089749


In [8]:
# --- 4. 최종 데이터 정렬 및 생성 ---
# 두 데이터셋에 공통으로 존재하는 환자 ID를 찾습니다.
common_patients = list(set(df_survival['PATIENT_ID']) & set(df_exp_transposed.index))

# 공통 환자 기준으로 두 데이터셋을 필터링하고 순서를 맞춥니다.
df_survival_final = df_survival.set_index('PATIENT_ID').loc[common_patients]
X_full = df_exp_transposed.loc[common_patients]

# 최종 y_full (Target)을 생성합니다.
y_full = df_survival_final['EVENT']

# 유전자 발현량 데이터의 결측치(NaN)를 0으로 채웁니다.
X_full = X_full.fillna(0)

In [9]:
y_full

,EVENT
PATIENT_ID,
TCGA-AB-2924,0
TCGA-AB-2835,0
TCGA-AB-2938,1
TCGA-AB-2857,1
TCGA-AB-2950,0
...,...
TCGA-AB-3008,1
TCGA-AB-2920,1
TCGA-AB-2908,1


In [10]:
X_full

Hugo_Symbol,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,REXO1L6P,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-AB-2924,5.698075,347.703423,2.204656,0.0,0.0,2.558526,4.003790,1.570317,0.0,0.0,...,422.412659,523.419729,54.676371,351.962844,3517.520897,5.185603,723.823718,12092.601846,3416.354316,1085.130191
TCGA-AB-2835,-0.436849,614.781393,5.799522,0.0,0.0,5.392561,11.027683,-0.268953,0.0,0.0,...,551.536561,1093.682228,42.855555,284.584147,2848.554124,0.301440,734.363684,13547.416047,4009.159561,654.129046
TCGA-AB-2938,55.299548,407.427399,3.150122,0.0,0.0,1.781595,7.002253,4.677906,0.0,0.0,...,733.722961,1291.586597,47.827280,321.011925,1968.605060,3.838961,864.761637,3183.254766,1877.636467,1571.772097
TCGA-AB-2857,27.177721,698.947257,1.633971,0.0,0.0,2.828415,5.425449,2.246909,0.0,0.0,...,643.543509,1590.644059,92.503665,514.199538,3169.573929,13.566166,1034.798280,2159.782958,2267.437082,1745.566625
TCGA-AB-2950,4.871366,503.819368,0.941057,0.0,0.0,4.276740,5.284053,1.403640,0.0,0.0,...,650.267397,845.862927,62.794290,422.376752,3678.990986,3.271647,660.336395,4580.551035,3744.716121,1140.808887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-AB-3008,15.335492,714.199146,0.087451,0.0,0.0,2.720137,7.148966,0.707090,0.0,0.0,...,794.867129,1591.091341,102.043613,419.006126,2807.095490,0.301440,817.559831,4022.122449,3537.666597,1030.996789
TCGA-AB-2920,2.124370,334.122615,2.487700,0.0,0.0,0.280166,5.615579,3.361335,0.0,0.0,...,267.723053,1107.989592,168.105183,571.460480,4947.803460,5.830998,512.472171,8904.881389,2271.642254,729.432545
TCGA-AB-2908,7.332828,422.030025,1.796986,0.0,0.0,-0.255074,3.797286,-0.268953,0.0,0.0,...,580.564375,1121.636900,111.426359,551.360040,2488.531389,5.233473,476.458066,7154.002849,1498.232642,1077.913961
TCGA-AB-2873,5.033071,419.398773,2.065170,0.0,0.0,10.158374,1.380428,-0.268953,0.0,0.0,...,430.847514,726.887069,42.321264,350.852076,3874.904604,4.867090,1264.716708,9205.674893,5257.714844,768.767364


In [11]:
print("--- 데이터 준비 완료 ---")
print("전체 Feature 데이터 (X_full) 형태:", X_full.shape)
print("전체 Target 데이터 (y_full) 형태:", y_full.shape)

--- 데이터 준비 완료 ---
전체 Feature 데이터 (X_full) 형태: (161, 20518)
전체 Target 데이터 (y_full) 형태: (161,)


In [12]:
pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=48af7d959a739e05a27b6d7c2a703a7a0ab4df16cddaa0340ed592730a11352d
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [13]:
from lifelines.statistics import logrank_test
import pandas as pd
from tqdm import tqdm # 루프 진행 상황을 보여주는 라이브러리

results = []

# tqdm을 사용하여 전체 유전자에 대한 반복문의 진행 상황을 확인합니다.
for gene in tqdm(X_full.columns, desc="Analyzing all genes (Log-rank)"):
    try:
        # 현재 유전자의 발현량 데이터
        gene_expression = X_full[gene]

        # 그룹 나누기
        median_expression = gene_expression.median()
        if median_expression == 0:
            continue

        high_group_filter = gene_expression >= median_expression
        low_group_filter = gene_expression < median_expression

        # 로그-순위 검정 수행
        result = logrank_test(
            durations_A=df_survival_final.loc[high_group_filter, 'OS_MONTHS'],
            durations_B=df_survival_final.loc[low_group_filter, 'OS_MONTHS'],
            event_observed_A=df_survival_final.loc[high_group_filter, 'EVENT'],
            event_observed_B=df_survival_final.loc[low_group_filter, 'EVENT']
        )

        # 결과 저장
        results.append({'gene': gene, 'p_value': result.p_value})

    except Exception as e:
        continue

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# p-value가 0.05 미만인 유의미한 결과만 필터링하고 정렬
significant_genes_logrank = results_df[results_df['p_value'] < 0.05].sort_values(by='p_value')

print("\n--- [방법 1] 생존율과 유의미한 관계가 있는 유전자 목록 (상위 20개) ---")
print(significant_genes_logrank.head(20))

Analyzing all genes (Log-rank): 100%|██████████| 20518/20518 [06:05<00:00, 56.13it/s]


--- [방법 1] 생존율과 유의미한 관계가 있는 유전자 목록 (상위 20개) ---
            gene       p_value
10912    PLA2G4A  3.373376e-07
7972        LNP1  5.170898e-07
2750       CCND3  6.492527e-07
10108     NUP210  1.222240e-06
8776      MBTPS1  1.586360e-06
11686     PWWP2A  1.739346e-06
14927    TOMM40L  2.134345e-06
9417       MYH15  2.334456e-06
14146      SYTL4  2.797985e-06
10421      PARP3  4.677992e-06
14086     SUZ12P  7.826799e-06
14216    TAS2R13  9.059458e-06
15038     TREML2  1.068948e-05
9802      NHLRC3  1.336326e-05
14441      TGIF1  2.433594e-05
8276   LOC646762  2.452533e-05
4125       DHRSX  2.733385e-05
3038      CENPC1  2.757660e-05
5410        FIBP  3.035758e-05
4077        DET1  3.141394e-05


In [25]:
significant_genes_logrank.head(100).style.format({'p_value': '{:.8f}'})

,gene,p_value
10912,PLA2G4A,0.00000034
7972,LNP1,0.00000052
2750,CCND3,0.00000065
10108,NUP210,0.00000122
8776,MBTPS1,0.00000159
11686,PWWP2A,0.00000174
14927,TOMM40L,0.00000213
9417,MYH15,0.00000233
14146,SYTL4,0.00000280
10421,PARP3,0.00000468
